<a href="https://colab.research.google.com/github/Nega0619/Aiffel_Submitted_Exploration_nodes/blob/main/Going_Deeper(NLP)_06_Investigate_the_biases_in_the_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/GoingDeeper(NLP)

/content/drive/MyDrive/GoingDeeper(NLP)


In [3]:
pwd

'/content/drive/MyDrive/GoingDeeper(NLP)'